# Embedding 模型比較

本範例展示：
1. **第1個儲存格**：使用不同 Embedding 模型建立向量資料庫
2. **第2個儲存格**：比較不同模型的檢索效果

學習目標：理解不同 Embedding 模型對檢索結果的影響，選擇最適合的模型

In [ ]:
# 第1個儲存格：使用不同 Embedding 模型建立向量資料庫

import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "三國演義.txt")
db_dir = os.path.join(current_dir, "db")

# 檢查文字檔案是否存在
if not os.path.exists(file_path):
    raise FileNotFoundError(
        f"檔案 {file_path} 不存在。請檢查路徑。"
    )

# 從檔案讀取文字內容
loader = TextLoader(file_path)
documents = loader.load()

# 將文件分割成塊
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 顯示分割文件的資訊
print("\n--- 文件塊資訊 ---")
print(f"文件塊數量: {len(docs)}")
print(f"範例塊:\n{docs[0].page_content}\n")


# 建立並持久化向量存儲的函數
def create_vector_store(docs, embeddings, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if not os.path.exists(persistent_directory):
        print(f"\n--- 正在建立向量存儲 {store_name} ---")
        Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory)
        print(f"--- 完成建立向量存儲 {store_name} ---")
    else:
        print(
            f"向量存儲 {store_name} 已存在。無需初始化。")


# 1. Jina Embeddings v2 (繁體中文專用)
# 專門為中英雙語優化的模型
# 維度：768，最大序列長度：8192 tokens
# 適用於：繁體中文內容、長文檔處理
print("\n--- 使用 Jina Embeddings v2（繁體中文專用）---")
jina_embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-zh"
)
create_vector_store(docs, jina_embeddings, "chroma_db_jina_nb")

# 2. Multilingual E5 Large (多語言模型)
# 微軟開發的多語言嵌入模型
# 維度：1024，最大序列長度：512 tokens
# 適用於：多語言內容、高精度需求
print("\n--- 使用 Multilingual E5 Large（多語言模型）---")
e5_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"
)
create_vector_store(docs, e5_embeddings, "chroma_db_e5_nb")

# 3. All-MiniLM-L6-v2 (輕量級英文模型)
# Sentence Transformers 的輕量級模型
# 維度：384，最大序列長度：256 tokens
# 適用於：速度優先、資源受限環境
print("\n--- 使用 All-MiniLM-L6-v2（輕量級模型）---")
minilm_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
create_vector_store(docs, minilm_embeddings, "chroma_db_minilm_nb")

# 4. All-MPNet-Base-v2 (平衡型英文模型)
# Sentence Transformers 的平衡型模型
# 維度：768，最大序列長度：384 tokens
# 適用於：英文內容、平衡效能與速度
print("\n--- 使用 All-MPNet-Base-v2（平衡型模型）---")
mpnet_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
create_vector_store(docs, mpnet_embeddings, "chroma_db_mpnet_nb")

print("\n=== 所有向量存儲已建立完成 ===")
print("\n模型特性比較：")
print("- Jina v2: 768維，8192 tokens，繁體中文專用")
print("- E5 Large: 1024維，512 tokens，多語言高精度")
print("- MiniLM: 384維，256 tokens，輕量快速")
print("- MPNet: 768維，384 tokens，英文平衡型")

In [ ]:
# 第2個儲存格：比較不同模型的檢索效果

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義路徑
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")

# 查詢向量存儲的函數
def query_vector_store(store_name, query, embedding_function):
    persistent_directory = os.path.join(db_dir, store_name)
    if os.path.exists(persistent_directory):
        print(f"\n{'='*60}")
        print(f"正在查詢向量存儲：{store_name}")
        print(f"{'='*60}")
        db = Chroma(
            persist_directory=persistent_directory,
            embedding_function=embedding_function,
        )
        retriever = db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": 3, "score_threshold": 0.1},
        )
        relevant_docs = retriever.invoke(query)
        
        # 顯示檢索結果數量
        print(f"\n找到 {len(relevant_docs)} 個相關文件\n")
        
        # 顯示相關結果及元數據
        for i, doc in enumerate(relevant_docs, 1):
            print(f"--- 文件 {i} ---")
            print(f"{doc.page_content[:200]}...")  # 只顯示前 200 字元
            if doc.metadata:
                print(f"來源: {doc.metadata.get('source', 'Unknown')}")
            print()
    else:
        print(f"向量存儲 {store_name} 不存在。")

# 定義不同的嵌入模型
jina_embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-zh"
)
e5_embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"
)
minilm_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
mpnet_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# 定義使用者的問題
query = "劉備是誰?"

print("\n" + "="*60)
print(f"查詢問題：{query}")
print("="*60)

# 查詢每個向量存儲並比較結果
query_vector_store("chroma_db_jina_nb", query, jina_embeddings)
query_vector_store("chroma_db_e5_nb", query, e5_embeddings)
query_vector_store("chroma_db_minilm_nb", query, minilm_embeddings)
query_vector_store("chroma_db_mpnet_nb", query, mpnet_embeddings)

print("\n" + "="*60)
print("結論：")
print("="*60)
print("1. Jina v2 和 E5 Large 對繁體中文的理解最佳")
print("2. MiniLM 和 MPNet 主要針對英文優化，中文效果較差")
print("3. 建議繁體中文應用使用 Jina v2 或 E5 Large")
print("4. 如需處理長文檔，Jina v2 (8192 tokens) 優於 E5 (512 tokens)")
print("="*60)